In [12]:
import yaml
from web3 import Web3
import json
import solcx
from solcx import install_solc

import json

install_solc("0.6.0")

Version('0.6.0')

# Importing Keys

In [9]:
with open('../keys.yml', 'r') as file:
    keys = yaml.safe_load(file)

# Connecting to a node

In [6]:
# Defining the URL of the node that will be interacting with the blockchain
infura_url = "HTTP://127.0.0.1:7545"

# Connecting the API to the node using the Web3 library
web3 = Web3(Web3.HTTPProvider(infura_url))

# Checking if the connection to the node was successful
if web3.isConnected():
    print("Successfully connected to the node at", infura_url)
else:
    print("Connection to node failed. Please check the URL and try again.")


Successfully connected to the node at HTTP://127.0.0.1:7545


In [11]:
# Getting the current block number of the entire blockchain
current_block_number = web3.eth.blockNumber
print("The current block number is", current_block_number)

# Defining the address of the wallet whose balance we want to check
wallet_address = keys["ganache_test1"]["public_key"]

# Getting the balance of the wallet in Wei using the getBalance method
balance_in_wei = web3.eth.getBalance(wallet_address)

# Converting the balance from Wei to Ether using the fromWei method
balance_in_ether = web3.fromWei(balance_in_wei, "ether")
print("The balance of the wallet at", wallet_address, "is", balance_in_ether, "Ether")


The current block number is 0
The balance of the wallet at 0x82dbbb95BEDB980E6bE1cCbc013B18c72af30149 is 100 Ether


# Defining Public and Private Key

In [33]:
# Defining the Ethereum addresses of two accounts
account_1 = keys["ganache_test1"]["public_key"]
account_2 = keys["ganache_test2"]["public_key"]

# Defining the private key of the first account
private_key = keys["ganache_test1"]["private_key"]

# Creating Smart Contract

In [14]:
with open("../smart_contract//Greeter.sol", "r") as file:
    source_code = file.read()

# Compile the contract
compiled_contract = solcx.compile_source(source_code)

# Get the ABI and bytecode of the contract
contract_abi = compiled_contract["<stdin>:Greeter"]["abi"]
contract_bytecode = compiled_contract["<stdin>:Greeter"]["bin"]

In [23]:
with open("../smart_contract//Greeter_abi.json", "w") as file:
    json.dump(contract_abi,file)
with open("../smart_contract//Greeter_bytecode.pkl", "w") as file:
    file.write(str(contract_bytecode))

In [25]:
# Deploy the contract
contract = web3.eth.contract(
    abi=contract_abi,
    bytecode=contract_bytecode
)



# Getting the nonce (number of transactions sent) of the first account
nonce = web3.eth.getTransactionCount(account_1)


# Get the transaction hash of the deployment
transaction = contract.constructor().build_transaction(
                                                        {
                                                            "gasPrice": web3.eth.gas_price,
                                                            "from" : account_1,
                                                            "nonce" : nonce
                                                        }

                                                        )

In [27]:
#sign transaction
signed_transaction = web3.eth.account.sign_transaction(transaction, private_key = private_key)

#transction hash
tx_hash = web3.eth.send_raw_transaction(signed_transaction.rawTransaction)

# Wait for the transaction to be mined
tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)

# Get the address of the deployed contract
contract_address = tx_receipt["contractAddress"]

# Create a contract instance using the address of the deployed contract
greeter = web3.eth.contract(
    address=contract_address,
    abi=contract_abi
)


In [28]:
contract_address

'0x903852a0fAbF43672D63da340ec58382e81d2430'

# Executing Smart Contract

In [29]:

# Defining the ABI and address of the contract
abi = ""  # Replace with the ABI of your contract

with open("../smart_contract//Greeter_abi.json","r") as file:
    abi = json.load(file)

contract_address = contract_address

address = web3.toChecksumAddress(contract_address)  # Replace with the address of your contract

# Creating a contract object using the Web3 library
contract = web3.eth.contract(address=address, abi=abi)

# Printing the current greeting in the contract
print(contract.functions.greet().call())



Hello


In [31]:

# Getting the nonce (number of transactions sent) of the first account
nonce = web3.eth.getTransactionCount(account_1)


# Get the transaction hash of the deployment
transaction = contract.functions.setGreeting('Got the concept!').build_transaction(
                                                                                {
                                                                                    "gasPrice": web3.eth.gas_price,
                                                                                    "from" : account_1,
                                                                                    "nonce" : nonce
                                                                                }

                                                                                )


#sign transaction
signed_transaction = web3.eth.account.sign_transaction(transaction, private_key = private_key)

#transction hash
tx_hash = web3.eth.send_raw_transaction(signed_transaction.rawTransaction)


# # Updating the greeting in the contract
# tx_hash = contract.functions.setGreeting('NEW GREETING!').transact()

# Waiting for the transaction to be mined
web3.eth.waitForTransactionReceipt(tx_hash)



AttributeDict({'transactionHash': HexBytes('0x8e09c4481ea9ad4c0900477a39e86939ac113c3e65ac994b89091b19cd6ca7dd'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x2e881212e96e1e154717e2d154e85cc7009bba67facd3fbd12f423abc9f32c21'),
 'blockNumber': 2,
 'from': '0x82dbbb95BEDB980E6bE1cCbc013B18c72af30149',
 'to': '0x903852a0fAbF43672D63da340ec58382e81d2430',
 'gasUsed': 29205,
 'cumulativeGasUsed': 29205,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000')

In [32]:
# Printing the updated greeting in the contract
print('Updated greeting: {}'.format(contract.functions.greet().call()))


Updated greeting: Got the concept!
